In [1]:
!pip install -q langchain\
                langgraph==1.0.5 \
                langchain-openai \
                langchain_experimental \
                aiosqlite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
!pip install -q langchain_community

In [3]:
!pip install -q python-dotenv==1.0.1 \
                openpipe-art==0.5.0 \
                langgraph-checkpoint-sqlite==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.8/767.8 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 

In [4]:
import subprocess

packages = ["langchain", "langgraph", "langchain-openai", "langchain_experimental" ]

for pkg in packages:
    # Install the package
    subprocess.run(["pip", "install", pkg])

    # Get the version from pip show
    result = subprocess.run(
        ["pip", "show", pkg],
        capture_output=True,
        text=True
    )

    version = None
    for line in result.stdout.splitlines():
        if line.startswith("Version:"):
            version = line.split(":", 1)[1].strip()
            break

    if version:
        print(f"{pkg}=={version}")
    else:
        print(f"Could not find version for {pkg}")

langchain==1.2.0
langgraph==1.0.5
langchain-openai==1.1.6
langchain_experimental==0.4.1


In [5]:
# Imports for API
from dotenv import load_dotenv
import os

load_dotenv()

NEBIUS_API_KEY = os.getenv('NEBIUS_API_KEY')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


# Imports

In [6]:
from typing import List, Optional, Literal, Annotated, TypedDict
import json
import uuid

from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver


In [7]:
base_llm = ChatOpenAI(
    model="Qwen/Qwen3-32B-fast",
    temperature=0,
    api_key=NEBIUS_API_KEY,
    base_url="https://api.studio.nebius.ai/v1/",
    extra_body={
        "top_k": 20,
        "chat_template_kwargs": {"enable_thinking": False},
    },
)

In [8]:
#writer_model = base_llm.with_structured_output(JobBody).bind(
#    temperature=cfg.temperature
#)

#style_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
#judge_llm = ChatOpenAI(model="gpt-4o", temperature=0.1)

judge_llm = ChatOpenAI(
    model="Qwen/Qwen3-32B-fast",
    temperature=0,
    api_key=NEBIUS_API_KEY,
    base_url="https://api.studio.nebius.ai/v1/",
    extra_body={
        "top_k": 20,
        "chat_template_kwargs": {"enable_thinking": False},
    },
)

style_llm = ChatOpenAI(
    model="google/gemma-2-9b-it-fast",
    temperature=0,
    api_key=NEBIUS_API_KEY,
    base_url="https://api.studio.nebius.ai/v1/",
    extra_body={
        "top_k": 20,
    },
)

In [9]:
class SkillItem(BaseModel):
    name: str = Field(..., description="Example: Java, Django, Kubernetes")
    category: Optional[str] = Field(
        default=None,
        description="Example: frontend, backend, cloud, database, devops, data",
    )
    level: Optional[str] = Field(
        default=None,
        description="Example: basic, intermediate, advanced, expert",
    )


class JobBody(BaseModel):
    job_description: str
    requirements: List[str]
    benefits: List[str]
    duties: List[str]
    summary: Optional[str]


In [ ]:
class JobGenerationConfig(BaseModel):
    language: Literal["en", "de"] = "en"

    formality: Literal["casual", "neutral", "formal"] = "neutral"
    company_type: Literal[
        "startup",
        "scaleup",
        "sme",
        "corporate",
        "public_sector",
        "social_sector",
        "agency",
        "consulting",
        "hospitality",
        "retail",
    ] = "scaleup"

    industry: Literal[
        "generic",
        "finance",
        "healthcare",
        "social_care",
        "public_it",
        "ai_startup",
        "ecommerce",
        "manufacturing",
    ] = "generic"

    seniority_label: Optional[Literal["intern", "junior", "mid", "senior", "lead", "principal"]] = None
    min_years_experience: Optional[int] = None
    max_years_experience: Optional[int] = None

    skills: List[SkillItem] = Field(default_factory=list)
    benefit_keywords: List[str] = Field(default_factory=list)

    @property
    def temperature(self) -> float:
        base = {
            "formal": 0.2,
            "neutral": 0.35,
            "casual": 0.55,
        }[self.formality]

        if self.company_type in ("startup", "hospitality", "retail"):
            base += 0.05
        elif self.company_type in ("public_sector", "social_sector"):
            base -= 0.05

        if self.seniority_label in ["senior", "lead", "principal"]:
            base -= 0.05
        elif self.seniority_label in ["intern", "junior"]:
            base += 0.05

        if self.industry in ["finance", "healthcare", "public_it"]:
            base -= 0.05
        elif self.industry in ["ai_startup", "ecommerce"]:
            base += 0.05

        base = max(0.1, min(base, 0.75))
        return base

    def with_industry_defaults(self) -> "JobGenerationConfig":
        cfg = self.model_copy(deep=True)

        if cfg.industry == "finance":
            if not cfg.benefit_keywords:
                cfg.benefit_keywords = [
                    "berufliche Vorsorge (BVG)",
                    "Weiterbildung im Bereich Finanzmarkt",
                    "Bonusregelung",
                    "hybrides Arbeiten",
                ]

        if cfg.industry == "healthcare":
            if not cfg.benefit_keywords:
                cfg.benefit_keywords = [
                    "Work Life Balance",
                    "betriebliche Gesundheitsförderung",
                    "sicherer Arbeitsplatz im Gesundheitswesen",
                ]

        if cfg.industry == "public_it":
            cfg.company_type = "public_sector"
            if not cfg.benefit_keywords:
                cfg.benefit_keywords = [
                    "Vereinbarkeit von Beruf und Familie",
                    "attraktive Sozialleistungen",
                    "sicheres Arbeitsumfeld im öffentlichen Dienst (Bund/Kanton)",
                ]

        if cfg.industry == "ai_startup":
            cfg.company_type = "startup"
            if not cfg.benefit_keywords:
                cfg.benefit_keywords = [
                    "Remote Work in der Schweiz",
                    "Mitarbeiterbeteiligung (Stock Options)",
                    "Weiterbildungsbudget für Konferenzen",
                    "modernes Büro im Stadtzentrum",
                ]

        if cfg.industry == "ecommerce" and not cfg.benefit_keywords:
            cfg.benefit_keywords = [
                "Mitarbeiterrabatte",
                "flexible Arbeitszeiten",
                "hybrides Arbeiten",
            ]

        if cfg.industry == "manufacturing" and not cfg.benefit_keywords:
            cfg.benefit_keywords = [
                "attraktive Schichtmodelle",
                "Zuschuss zu Fahrtkosten",
                "berufliche Vorsorge (BVG)",
            ]

        return cfg


In [ ]:
from typing import List
import re
from langchain_openai import ChatOpenAI

from langchain_core.messages import AIMessage
# note -> import JobGenerationConfig, JobBody, JobState
# note -> import import explain_temperature

#base_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# ── Schweizer Schriftdeutsch helpers (inline for notebook) ──

def _replace_eszett(text: str) -> str:
    """Replace every ß with ss.  Swiss German never uses Eszett."""
    return text.replace("ß", "ss")

_CH_TERM_MAP = [
    (r"\bTarifvertrag\b", "Gesamtarbeitsvertrag (GAV)"),
    (r"\bGehalt\b", "Salär"),
    (r"\bGehalts\b", "Salärs"),
    (r"\bGehaltsvorstellung\b", "Salärvorstellung"),
    (r"\bGehaltsabrechnung\b", "Lohnabrechnung"),
    (r"\bArbeitnehmer\b", "Arbeitnehmende"),
    (r"\bAbitur\b", "Matura"),
    (r"\bUrlaub\b", "Ferien"),
    (r"\bUrlaubstage\b", "Ferientage"),
    (r"\bbetriebliche Altersvorsorge\b", "berufliche Vorsorge (BVG)"),
    (r"\bBetriebsrente\b", "Pensionskasse (BVG)"),
    (r"\bFahrrad\b", "Velo"),
    (r"\bStraßenbahn\b", "Tram"),
]

def _enforce_swiss_german(text: str) -> str:
    """ß→ss + DE-DE→DE-CH vocabulary."""
    text = _replace_eszett(text)
    for pattern, replacement in _CH_TERM_MAP:
        text = re.sub(pattern, replacement, text)
    return text

def _enforce_swiss_german_on_list(items: List[str]) -> List[str]:
    return [_enforce_swiss_german(item) for item in items]

CH_SCHRIFTDEUTSCH_PROMPT_BLOCK = """\
## Schweizer Schriftdeutsch (OBLIGATORISCH)

Du schreibst AUSSCHLIESSLICH in Schweizer Schriftdeutsch (CH-Deutsch).
Beachte folgende Regeln STRIKT:

### Orthografie
- NIEMALS «ß» verwenden. Immer «ss» schreiben (z.B. «gross» statt «groß», «Strasse» statt «Straße»).

### Wortschatz (CH-spezifisch)
- «Salär» statt «Gehalt»
- «Gesamtarbeitsvertrag (GAV)» statt «Tarifvertrag»
- «Ferien» statt «Urlaub»
- «Matura» statt «Abitur»
- «berufliche Vorsorge (BVG)» statt «betriebliche Altersvorsorge»
- «Pensionskasse» statt «Betriebsrente»
- «Arbeitnehmende» statt «Arbeitnehmer»

### Stilistische Merkmale
- Neutraler, sachlicher Ton (weniger werblich als DE-DE)
- Etwas kürzere Sätze als in Deutschland üblich
- Weniger rhetorischer Schmuck, weniger Ausrufezeichen
- Prozedurale Klarheit hat Vorrang vor emotionaler Ansprache
"""


def render_job_body(
    job_title: str,
    cfg: JobGenerationConfig,
    temperature: float | None = None,
) -> JobBody:
    """
    Pure JD generator.
    Builds the prompt from JobGenerationConfig and returns a JobBody instance.
    No LangGraph state, no messages list, no JSON.
    German output always uses Schweizer Schriftdeutsch (no ß, CH vocabulary).
    """

    cfg = cfg.with_industry_defaults()
    lang = cfg.language
    temp = temperature if temperature is not None else cfg.temperature

    writer_model = base_llm.with_structured_output(JobBody).bind(
        temperature=temp
    )

    # tone line (German: includes explicit Sie/du pronoun rule)
    if lang == "en":
        tone_map = {
            "casual": "Use a friendly modern tone, but stay professional.",
            "neutral": "Use a clear neutral professional tone.",
            "formal": "Use a formal corporate tone.",
        }
    else:
        tone_map = {
            "casual": (
                "Verwende einen freundlichen modernen, aber professionellen Ton. "
                "WICHTIG: Verwende durchgehend die «du»-Anrede (duzen). "
                "Beispiel: «Du arbeitest…», «Dein Team…», «Wir bieten dir…». "
                "Verwende NIEMALS «Sie» oder «Ihnen» als Höflichkeitsform."
            ),
            "neutral": (
                "Verwende einen klaren sachlich professionellen Ton. "
                "WICHTIG: Verwende durchgehend die «Sie»-Anrede (siezen). "
                "Beispiel: «Sie arbeiten…», «Ihr Team…», «Wir bieten Ihnen…». "
                "Verwende NIEMALS «du» oder «dir» als Anrede."
            ),
            "formal": (
                "Verwende einen formellen, eher konservativen Ton. "
                "WICHTIG: Verwende durchgehend die «Sie»-Anrede (siezen). "
                "Beispiel: «Sie arbeiten…», «Ihr Team…», «Wir bieten Ihnen…». "
                "Verwende NIEMALS «du» oder «dir» als Anrede."
            ),
        }
    tone_line = tone_map[cfg.formality]

    # company type line
    if lang == "en":
        type_map = {
            "startup": "The company is a young startup with a fast paced environment.",
            "scaleup": "The company is a growing scaleup with an established product.",
            "sme": "The company is a small or medium-sized enterprise (SME) with close-knit teams and a pragmatic work culture.",
            "corporate": "The company is a larger established company.",
            "public_sector": "The organization operates in the public sector.",
            "social_sector": "The organization is a non-profit or social sector entity with a purpose-driven mission.",
            "agency": "The company is a digital agency working for multiple clients.",
            "consulting": "The company is a consulting firm that delivers client projects.",
            "hospitality": "The company operates in the hospitality sector (e.g. restaurant franchise, gastronomy) with a hands-on, team-oriented culture.",
            "retail": "The company operates in retail with a customer-focused, dynamic work environment.",
        }
    else:
        type_map = {
            "startup": "Das Unternehmen ist ein junges Startup mit dynamischem Umfeld.",
            "scaleup": "Das Unternehmen ist ein wachsendes Scaleup mit etabliertem Produkt.",
            "sme": "Das Unternehmen ist ein KMU mit eingespielten Teams und einer pragmatischen Arbeitskultur.",
            "corporate": "Das Unternehmen ist ein grösseres etabliertes Unternehmen.",
            "public_sector": "Die Organisation ist im öffentlichen Sektor tätig.",
            "social_sector": "Die Organisation ist eine gemeinnützige Einrichtung oder Stiftung mit einer sinnstiftenden Mission.",
            "agency": "Das Unternehmen ist eine Agentur mit verschiedenen Kundenprojekten.",
            "consulting": "Das Unternehmen ist ein Beratungsunternehmen mit vielfältigen Kundenprojekten.",
            "hospitality": "Das Unternehmen ist in der Gastronomie tätig (z.B. Restaurantkette) mit einer praxisnahen, teamorientierten Kultur.",
            "retail": "Das Unternehmen ist im Detailhandel tätig mit einem kundenorientierten, dynamischen Arbeitsumfeld.",
        }
    company_line = type_map[cfg.company_type]

    # seniority line
    seniority_bits: List[str] = []
    if cfg.seniority_label:
        if lang == "en":
            seniority_bits.append(f"The role is a {cfg.seniority_label} level position.")
        else:
            seniority_bits.append(f"Die Rolle ist auf {cfg.seniority_label} Level ausgerichtet.")
    if cfg.min_years_experience is not None:
        if lang == "en":
            if cfg.max_years_experience:
                seniority_bits.append(
                    f"Target experience range is {cfg.min_years_experience} to {cfg.max_years_experience} years."
                )
            else:
                seniority_bits.append(
                    f"Target experience is at least {cfg.min_years_experience} years."
                )
        else:
            if cfg.max_years_experience:
                seniority_bits.append(
                    f"Die gewünschte Erfahrung liegt zwischen {cfg.min_years_experience} und {cfg.max_years_experience} Jahren."
                )
            else:
                seniority_bits.append(
                    f"Gesucht werden Kandidatinnen und Kandidaten mit mindestens {cfg.min_years_experience} Jahren Berufserfahrung."
                )
    seniority_line = " ".join(seniority_bits)

    # skills line
    skills_text = ", ".join([s.name for s in cfg.skills]) if cfg.skills else ""
    if lang == "en":
        skills_line = (
            f"Required core skills: {skills_text}."
            if skills_text
            else "Infer reasonable skills for this job title and industry."
        )
    else:
        skills_line = (
            f"Zentrale Skills: {skills_text}."
            if skills_text
            else "Ergänze sinnvolle Skills passend zu Titel und Branche."
        )

    # benefits line
    benefit_tags = ", ".join(cfg.benefit_keywords) if cfg.benefit_keywords else ""
    if lang == "en":
        benefits_line = (
            "Use these benefit keywords and turn them into nicely written bullet points: "
            f"{benefit_tags}."
            if benefit_tags
            else "Propose a realistic set of benefits for this industry and company type."
        )
    else:
        benefits_line = (
            "Nutze diese Benefit Stichworte und formuliere daraus ansprechende Bullet Points: "
            f"{benefit_tags}."
            if benefit_tags
            else "Schlage ein realistisches Paket an Benefits für diese Branche und diesen Unternehmenstyp vor."
        )

    # CH prompt block for German generation
    ch_block = ""
    if lang == "de":
        ch_block = CH_SCHRIFTDEUTSCH_PROMPT_BLOCK + "\n"

    # ── Sentence-start variety (AGENTS.md Rule 5) ──
    variety_en = (
        "\n## Sentence-Start Variety (MANDATORY)\n"
        "Every bullet-point list (duties, requirements, benefits) MUST vary how sentences begin.\n"
        "NEVER start more than 2 bullets the same way (e.g. 'You will…', 'Be responsible for…').\n"
        "Mix these openings across each list:\n"
        "- Lead with the topic area (e.g. 'Cloud infrastructure: design and maintain…')\n"
        "- Start with an action verb (e.g. 'Design scalable APIs…')\n"
        "- Open with context (e.g. 'In close collaboration with the data team, …')\n"
        "- Use a noun phrase (e.g. 'Ownership of the CI/CD pipeline…')\n"
        "- Vary the subject: you / we / the team / this role\n"
    )
    variety_de = (
        "\n## Abwechslung bei Satzanfängen (OBLIGATORISCH)\n"
        "Jede Aufzählung (Aufgaben, Anforderungen, Benefits) MUSS unterschiedliche Satzanfänge verwenden.\n"
        "NIEMALS mehr als 2 Punkte gleich beginnen (z.B. immer «Sie …» oder «Du …»).\n"
        "Verwende mindestens 3 dieser Varianten pro Liste:\n"
        "- Mit dem Themenbereich beginnen (z.B. «Cloud-Infrastruktur: Entwurf und Wartung…»)\n"
        "- Mit einem Verb starten (z.B. «Entwirf skalierbare APIs…» oder «Skalierbare APIs entwerfen…»)\n"
        "- Kontextbezogener Einstieg (z.B. «In enger Zusammenarbeit mit dem Data-Team…»)\n"
        "- Substantivische Wendung (z.B. «Verantwortung für die CI/CD-Pipeline…»)\n"
        "- Subjekt wechseln: Sie/Du / Wir / Das Team / Diese Rolle\n"
    )

    # prompt
    if lang == "en":
        prompt = (
            "You are an experienced HR copywriter for a recruitment platform.\n"
            f"{tone_line}\n"
            f"{company_line}\n"
            f"{seniority_line}\n"
            f"{skills_line}\n"
            f"{benefits_line}\n"
            f"{variety_en}\n"
            f"Job title: {job_title}\n\n"
            "Produce a JobBody instance in English.\n"
            "job_description: 2 to 4 sentences for role and context.\n"
            "requirements: 6 to 10 bullets matching seniority and skills.\n"
            "benefits: 4 to 8 bullets, phrasing the benefit keywords naturally.\n"
            "duties: 5 to 8 bullets describing day to day responsibilities.\n"
            "summary: 1 short closing line inviting candidates to apply.\n"
        )
    else:
        prompt = (
            "Du bist eine erfahrene HR Texterin für eine Recruiting Plattform.\n"
            f"{ch_block}"
            f"{tone_line}\n"
            f"{company_line}\n"
            f"{seniority_line}\n"
            f"{skills_line}\n"
            f"{benefits_line}\n"
            f"{variety_de}\n"
            f"Stellentitel: {job_title}\n\n"
            "Erstelle eine JobBody Struktur auf Schweizer Schriftdeutsch.\n"
            "job_description: 2 bis 4 Sätze zu Rolle und Kontext.\n"
            "requirements: 6 bis 10 Stichpunkte, passend zur Seniorität und zu den Skills.\n"
            "benefits: 4 bis 8 Stichpunkte, formuliere die Benefit Stichworte aus.\n"
            "duties: 5 bis 8 Stichpunkte zu den täglichen Aufgaben.\n"
            "summary: 1 kurzer Abschlusssatz, der zur Bewerbung einlädt.\n"
        )

    payload: JobBody = writer_model.invoke(prompt)

    # ── Schweizer Schriftdeutsch post-processing (ß→ss + CH vocabulary) ──
    if lang == "de":
        payload.job_description = _enforce_swiss_german(payload.job_description)
        payload.requirements = _enforce_swiss_german_on_list(payload.requirements)
        payload.benefits = _enforce_swiss_german_on_list(payload.benefits)
        payload.duties = _enforce_swiss_german_on_list(payload.duties)
        if payload.summary:
            payload.summary = _enforce_swiss_german(payload.summary)

    return payload


In [ ]:
def explain_temperature(cfg: JobGenerationConfig) -> str:
    parts = []

    base_map = {"formal": 0.2, "neutral": 0.35, "casual": 0.55}
    base = base_map[cfg.formality]
    parts.append(f"Base from formality='{cfg.formality}': {base:.2f}")

    # company type
    if cfg.company_type in ("startup", "hospitality", "retail"):
        base += 0.05
        parts.append(f"Company type '{cfg.company_type}' => +0.05")
    elif cfg.company_type in ("public_sector", "social_sector"):
        base -= 0.05
        parts.append(f"Company type '{cfg.company_type}' => -0.05")
    else:
        parts.append(f"Company type '{cfg.company_type}' => +0.00")

    # seniority
    if cfg.seniority_label in ["senior", "lead", "principal"]:
        base -= 0.05
        parts.append(f"Seniority '{cfg.seniority_label}' => -0.05")
    elif cfg.seniority_label in ["intern", "junior"]:
        base += 0.05
        parts.append(f"Seniority '{cfg.seniority_label}' => +0.05")
    else:
        parts.append("Seniority not set => +0.00")

    # industry
    if cfg.industry in ["finance", "healthcare", "public_it"]:
        base -= 0.05
        parts.append(f"Industry '{cfg.industry}' => -0.05")
    elif cfg.industry in ["ai_startup", "ecommerce"]:
        base += 0.05
        parts.append(f"Industry '{cfg.industry}' => +0.05")
    else:
        parts.append(f"Industry '{cfg.industry}' => +0.00")

    clamped = max(0.1, min(base, 0.75))
    if clamped != base:
        parts.append(f"Clamped to [0.10, 0.75] => {clamped:.2f}")
    else:
        parts.append(f"Final temperature => {clamped:.2f}")

    return "\n".join(parts)


In [13]:
from typing import Annotated, TypedDict, List, Optional, Dict, Any, Literal
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage

def merge_blackboard(
    left: Optional[List[JobBody]],
    right: Optional[List[JobBody]],
) -> List[JobBody]:
    if right is None:
        return left or []
    return right

class JobState(TypedDict, total=False):
    messages: Annotated[List[BaseMessage], add_messages]
    job_title: str
    config: JobGenerationConfig
    company_urls: List[str]
    scraped_text: Optional[str]

    candidates: Annotated[List[JobBody], merge_blackboard]

    job_body_json: Optional[str]
    style_profile_json: Optional[str]
    consistency_report_json: Optional[str]

    ruler_run: Dict[str, Any]
    ruler_runs: List[Dict[str, Any]]
    feedback_label: Literal["accepted", "rejected", "edited", "no_feedback"]
    user_feedback: Optional[str]


In [14]:
def generate_job_body(state: JobState) -> JobState:
    cfg = state["config"].with_industry_defaults()

    debug_info = explain_temperature(cfg)
    print("\n[JobGenerationConfig temperature breakdown]")
    print(debug_info)
    print()

    jb = render_job_body(
        job_title=state["job_title"],
        cfg=cfg,
        temperature=cfg.temperature,
    )

    state["job_body_json"] = jb.model_dump_json(indent=2, ensure_ascii=False)
    state["messages"].append(AIMessage(content=state["job_body_json"]))
    return state


In [15]:
import json
from typing import List, Tuple

import art
from art.rewards import ruler_score_group
from openai.types.chat.chat_completion import Choice
from openai.types.chat import ChatCompletionMessage


def generate_job_body_candidate(
    job_title: str,
    cfg: JobGenerationConfig,
    temp_jitter: float = 0.0,
) -> JobBody:
    """Use the same logic as the graph, with a slightly adjusted temperature."""
    base_temp = cfg.temperature
    temp = max(0.1, min(base_temp + temp_jitter, 0.9))
    return render_job_body(job_title, cfg, temperature=temp)


def jd_candidate_to_trajectory(
    job_title: str,
    cfg: JobGenerationConfig,
    job_body: JobBody,
) -> art.Trajectory:
    """
    Wrap a JobBody candidate as a trajectory for RULER.
    Messages:
      system: what the judge should care about
      user: config and job body
      assistant: the job body content that is being judged
    """

    system_msg = {
        "role": "system",
        "content": (
            "You are an expert HR quality judge. You evaluate job descriptions for clarity, "
            "tone, alignment with the requested role, and usefulness to candidates. "
            "You prefer job ads that are specific, concise, aligned with the seniority level, "
            "and realistic for the company type and industry."
        ),
    }

    user_msg = {
        "role": "user",
        "content": (
            "Evaluate the quality of the following job description.\n\n"
            f"Job title: {job_title}\n\n"
            f"Config JSON:\n{cfg.model_dump_json(indent=2, ensure_ascii=False)}\n\n"
            f"JobBody JSON:\n{job_body.model_dump_json(indent=2, ensure_ascii=False)}\n"
        ),
    }

    assistant_msg = ChatCompletionMessage(
        role="assistant",
        content=job_body.model_dump_json(indent=2, ensure_ascii=False),
    )

    choice = Choice(
        finish_reason="stop",
        index=0,
        message=assistant_msg,
    )

    traj = art.Trajectory(
        messages_and_choices=[system_msg, user_msg, choice],
        reward=0.0,
    )
    return traj


In [16]:
async def generate_best_job_body_with_ruler(
    job_title: str,
    cfg: JobGenerationConfig,
    num_candidates: int = 3,
    jitter: float = 0.1,
    judge_model: str = "openai/o3-mini",
) -> Tuple[JobBody, List[Tuple[float, JobBody]]]:
    """
    Generate multiple JobBody candidates and use ART RULER to pick the best one.

    Returns:
      best_job_body,
      list of (score, job_body) sorted by score descending.
    """

    # 1sample candidates with small temperature jitter
    candidates: List[JobBody] = []
    if num_candidates <= 0:
        raise ValueError("num_candidates must be at least 1")

    offsets: List[float] = []
    center = num_candidates // 2
    for i in range(num_candidates):
        offsets.append((i - center) * jitter)

    for offset in offsets:
        jb = generate_job_body_candidate(job_title, cfg, temp_jitter=offset)
        candidates.append(jb)

    # wrap as trajectories
    trajectories = [
        jd_candidate_to_trajectory(job_title, cfg, jb)
        for jb in candidates
    ]
    group = art.TrajectoryGroup(trajectories)

    # score with RULER
    judged_group = await ruler_score_group(group, judge_model, debug=False)
    if not judged_group:
        # graceful fallback
        return candidates[0], [(0.0, jb) for jb in candidates]

    # 4. collect scores and sort
    scored: List[Tuple[float, JobBody]] = []
    for traj, jb in zip(judged_group.trajectories, candidates):
        scored.append((traj.reward, jb))

    scored_sorted = sorted(scored, key=lambda t: t[0], reverse=True)
    best_score, best_job_body = scored_sorted[0]

    print("\n[JD RULER ranking]")
    for rank, (score, jb) in enumerate(scored_sorted, start=1):
        print(f"Rank {rank} | score={score:.3f}")
        print("  job_description:", (jb.job_description or "")[:140], "...")
    print()

    return best_job_body, scored_sorted


In [17]:
from __future__ import annotations

import hashlib
import sqlite3
import uuid
from datetime import datetime, timezone
from typing import Annotated, Any, Dict, List, Optional, TypedDict, Literal

from langchain_core.messages import AIMessage, BaseMessage
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.store.base import BaseStore
from langgraph.store.memory import InMemoryStore


# Your imports
# from your_project.models import JobGenerationConfig, JobBody, SkillItem
# from your_project.jd_generator import render_job_body
# from your_project.ruler import generate_best_job_body_with_ruler
# from your_project.style import build_style_profile, build_consistency_report
# from your_project.scrape import scrape_company_urls


def merge_blackboard(
    left: Optional[List[JobBody]],
    right: Optional[List[JobBody]],
) -> List[JobBody]:
    if right is None:
        return left or []
    return right


class JobState(TypedDict, total=False):
    messages: Annotated[List[BaseMessage], add_messages]
    job_title: str
    config: "JobGenerationConfig"
    company_urls: List[str]
    scraped_text: Optional[str]

    # IMPORTANT: keep this in the state schema
    candidates: Annotated[List[JobBody], merge_blackboard]

    job_body_json: Optional[str]
    style_profile_json: Optional[str]
    consistency_report_json: Optional[str]

    ruler_run: Dict[str, Any]
    ruler_runs: List[Dict[str, Any]]
    feedback_label: Literal["accepted", "rejected", "edited", "no_feedback"]
    user_feedback: Optional[str]


def _sha16(text: str) -> str:
    return hashlib.sha256(text.encode("utf-8")).hexdigest()[:16]


def _utc_now() -> str:
    return datetime.now(timezone.utc).isoformat()


def _bucket(cfg: "JobGenerationConfig") -> str:
    return "|".join(
        [
            cfg.language,
            cfg.industry,
            cfg.formality,
            cfg.company_type,
            cfg.seniority_label or "none",
        ]
    )


def _ruler_namespace(config: RunnableConfig) -> tuple[str, str]:
    user_id = config["configurable"]["user_id"]
    return (user_id, "jd_ruler")


async def node_scrape_company(state: JobState) -> JobState:
    """
    Optional stub. Keep it if you already scrape.
    This should populate scraped_text and possibly company_urls.
    """
    # scraped = await scrape_company_urls(state["company_urls"])
    # state["scraped_text"] = scraped.text
    return {"scraped_text": "Company info text..."}


def node_build_style_profile(state: JobState) -> JobState:
    """
    Optional stub. Keep it if you already produce style_profile_json.
    """
    # style = build_style_profile(state)
    # state["style_profile_json"] = style.model_dump_json(indent=2, ensure_ascii=False)
    return state

async def node_generator_expert(state: JobState, config: RunnableConfig, *, store: BaseStore) -> Dict:
    """Expert: Initial Drafter. Pulls 'Gold Standards' from SQLite Store."""
    user_id = config["configurable"].get("user_id", "default")
    cfg = state["config"].with_industry_defaults()

    # Access Shared Memory
    past_gold = store.search((user_id, "gold_standard"), query=state["job_title"], limit=2)

    seeds = []
    for i in range(3):
        jb = generate_job_body_candidate(state["job_title"], cfg, temp_jitter=(i * 0.1))
        seeds.append(jb)

    return {"candidates": seeds}

async def node_style_expert(state: JobState, config: RunnableConfig, *, store: BaseStore) -> Dict:
    """Expert: Refiner. Pulls 'User Gripes' from SQLite Store to polish the blackboard."""
    user_id = config["configurable"].get("user_id", "default")
    candidates = state.get("candidates", [])

    # Access Shared Memory
    gripes = store.search((user_id, "user_gripes"), limit=3)
    avoid_list = "\n".join([f"- {g.value['feedback']}" for g in gripes])

    # Creative Refinement: Usually you would call style_llm here
    # We ensure we return the candidates so they stay on the blackboard
    return {"candidates": candidates, "is_refined": True}

async def node_ruler_curator(state: JobState) -> Dict:
    """Expert: Final Judge. Selects the winner from the blackboard."""
    candidates = state.get("candidates", [])
    if not candidates:
        return {"job_body_json": "Error: Blackboard was empty at Curate step."}

    # RULER ranking logic
    trajectories = [jd_candidate_to_trajectory(state["job_title"], state["config"], jb) for jb in candidates]
    group = art.TrajectoryGroup(trajectories)
    judged_group = await art.rewards.ruler_score_group(group, "openai/o3-mini")

    if not judged_group:
        # Graceful fallback: use first candidate
        best_jb = candidates[0]
        return {
            "job_body_json": best_jb.model_dump_json(indent=2, ensure_ascii=False),
            "ruler_run": {"best_score": 0.0, "fallback": True}
        }

    scored = sorted(zip(judged_group.trajectories, candidates), key=lambda x: x[0].reward, reverse=True)
    best_jb = scored[0][1]

    return {
        "job_body_json": best_jb.model_dump_json(indent=2, ensure_ascii=False),
        "ruler_run": {"best_score": float(scored[0][0].reward)}
    }

def node_build_consistency_report(state: JobState) -> JobState:
    """
    Optional stub. If you already have a consistency report, keep it.
    This should evaluate job_body_json against style_profile_json and scraped_text.
    """
    # report = build_consistency_report(state)
    # state["consistency_report_json"] = report.model_dump_json(indent=2, ensure_ascii=False)
    return state


import aiosqlite
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.store.memory import InMemoryStore
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph, START, END


def node_persist_feedback_to_store(state: JobState, config: RunnableConfig, *, store: BaseStore) -> Dict:
    user_id = config["configurable"]["user_id"]
    final_body = state.get("job_body_json")

    if state.get("feedback_label") == "accepted" and final_body:
        # Save to SQLite for future 'generator' runs
        store.put((user_id, "gold_standard"), state["job_title"], {"body": final_body})

    return {} # Side-effect node

async def build_graph(*, sqlite_path: str = "jd_threads.sqlite"):
    workflow = StateGraph(JobState)

    workflow.add_node("scrape_company", node_scrape_company)
    workflow.add_node("generator", node_generator_expert)
    workflow.add_node("style_expert", node_style_expert)
    workflow.add_node("curator", node_ruler_curator)
    workflow.add_node("persist", node_persist_feedback_to_store)

    workflow.add_edge(START, "scrape_company")
    workflow.add_edge("scrape_company", "generator")
    workflow.add_edge("generator", "style_expert")
    workflow.add_edge("style_expert", "curator")
    workflow.add_edge("curator", "persist")
    workflow.add_edge("persist", END)

    conn = await aiosqlite.connect(sqlite_path)
    if not hasattr(conn, "is_alive"): conn.is_alive = lambda: True

    checkpointer = AsyncSqliteSaver(conn)
    store = InMemoryStore() # Use a persistent Store for production

    return workflow.compile(checkpointer=checkpointer, store=store), conn, store



In [18]:
import json
import uuid
from datetime import datetime, timezone
from pathlib import Path
from typing import Any, Dict, Optional, List

from langchain_core.runnables import RunnableConfig


def _utc_now_compact() -> str:
    return datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")


def _ensure_dir(p: Path) -> None:
    p.mkdir(parents=True, exist_ok=True)


def _json_dump(path: Path, obj: Any) -> None:
    _ensure_dir(path.parent)
    path.write_text(json.dumps(obj, ensure_ascii=False, indent=2), encoding="utf-8")


def _jsonl_append(path: Path, obj: Any) -> None:
    _ensure_dir(path.parent)
    with path.open("a", encoding="utf-8") as f:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")


def _safe_excerpt(s: Optional[str], n: int) -> Optional[str]:
    if not s:
        return None
    return s[:n] + ("..." if len(s) > n else "")


def _artifact_delta(payload: Any) -> Dict[str, Any]:
    """Defensive tracker for Blackboard changes during streaming."""
    # Ensure payload is a dictionary before calling .get()
    if not isinstance(payload, dict):
        return {"info": "Node returned non-dict payload"}

    candidates = payload.get("candidates", [])

    # Safely get the description of the first candidate if it exists
    first_desc = None
    if candidates and len(candidates) > 0:
        # Check if it's a JobBody object or a dict
        first_obj = candidates[0]
        first_desc = getattr(first_obj, "job_description", None)

    return {
        "candidate_count": len(candidates),
        "first_candidate_excerpt": _safe_excerpt(first_desc, 200),
        "job_body_json_excerpt": _safe_excerpt(payload.get("job_body_json"), 800),
        "ruler_run": payload.get("ruler_run"),
        "is_refined": payload.get("is_refined"),
    }


from typing import Any, Dict, List, Optional
from pydantic import BaseModel
from langchain_core.messages import BaseMessage


def _message_to_jsonable(m: BaseMessage) -> Dict[str, Any]:
    d: Dict[str, Any] = {
        "type": m.__class__.__name__,
        "content": getattr(m, "content", None),
    }

    # Some message types have role / name
    role = getattr(m, "role", None)
    if role is not None:
        d["role"] = role

    name = getattr(m, "name", None)
    if name is not None:
        d["name"] = name

    # Tool calls show up on AIMessage in many setups
    tool_calls = getattr(m, "tool_calls", None)
    if tool_calls:
        d["tool_calls"] = tool_calls

    additional_kwargs = getattr(m, "additional_kwargs", None)
    if additional_kwargs:
        d["additional_kwargs"] = additional_kwargs

    response_metadata = getattr(m, "response_metadata", None)
    if response_metadata:
        d["response_metadata"] = response_metadata

    return d


def _to_jsonable(obj: Any) -> Any:
    if isinstance(obj, BaseMessage):
        return _message_to_jsonable(obj)
    if isinstance(obj, BaseModel):
        return obj.model_dump()
    if isinstance(obj, dict):
        return {k: _to_jsonable(v) for k, v in obj.items()}
    if isinstance(obj, (list, tuple)):
        return [_to_jsonable(v) for v in obj]
    return obj


def _state_to_jsonable(state: Dict[str, Any]) -> Dict[str, Any]:
    s = dict(state)
    if "config" in s:
        s["config"] = _to_jsonable(s["config"])
    if "messages" in s and isinstance(s["messages"], list):
        s["messages"] = [_message_to_jsonable(m) for m in s["messages"]]
    return _to_jsonable(s)



async def run_example():
    graph, conn, store = await build_graph(sqlite_path="jd_threads.sqlite")

    run_id = f"{_utc_now_compact()}_{uuid.uuid4().hex[:8]}"
    run_dir = Path("artifacts") / run_id
    events_path = run_dir / "events.jsonl"

    try:
        initial_state: JobState = {
            "messages": [],
            "job_title": "Senior AI Agent Engineer",
            "candidates": [],
            "config": JobGenerationConfig(
                language="en",
                industry="ai_startup",
                formality="casual",
                company_type="scaleup",
                seniority_label="senior",
                min_years_experience=5,
                skills=[
                    SkillItem(name="Python", category="backend", level="advanced"),
                    SkillItem(name="LLM orchestration (LangGraph, LangChain)", category="backend"),
                ],
                benefit_keywords=["hybrid work in Switzerland", "continuing education in AI and ML"],
            ),
            "company_urls": [
                "https://www.refline.io/en/index.html",
                "https://www.refline.io/en/aboutus.html",
                "https://www.refline.io/en/career.html",
            ],
            "job_body_json": None,
            "style_profile_json": None,
            "consistency_report_json": None,
            "scraped_text": None,
            "feedback_label": "no_feedback",
            "user_feedback": None,
        }

        config: RunnableConfig = {
            "configurable": {"thread_id": "thread_001", "user_id": "user_123"}
        }

        # persist inputs for reproducibility
        _json_dump(
            run_dir / "input.json",
            {
                "run_id": run_id,
                "thread_id": config["configurable"]["thread_id"],
                "user_id": config["configurable"]["user_id"],
                "initial_state": _state_to_jsonable(initial_state),
            },
        )

        # stream updates and also produce a final result
        async for update in graph.astream(initial_state, config=config, stream_mode="updates"):
            for node, payload in update.items():
                record = {
                    "ts": datetime.now(timezone.utc).isoformat(),
                    "node": node,
                    "delta": _artifact_delta(payload),
                }
                _jsonl_append(events_path, record)

                print(f"\n[Expert Active: {node}]")
                d = record["delta"]

                if d.get("candidate_count"):
                    print(f"  Blackboard Update: {d['candidate_count']} candidates present.")
                    print(f"  Sample Draft: {d['first_candidate_excerpt']}")

                if d.get("is_refined"):
                    print("  ✨ Style Expert finished polishing candidates based on user history.")

                if d.get("job_body_json_excerpt"):
                    print(f"  Curator selected final candidate (Size: {len(payload['job_body_json'])} chars)")

                if d.get("ruler_run"):
                    rr = d["ruler_run"]
                    print(f"  RULER Ranking: Best Score = {rr.get('best_score')}")

        # IMPORTANT: async state accessors with AsyncSqliteSaver
        latest = await graph.aget_state(config)
        history = [snap async for snap in graph.aget_state_history(config)]

        final_values = latest.values
        _json_dump(run_dir / "final_state_values.json", _state_to_jsonable(final_values))


        # optional: write big blobs separately for convenience
        # Ensure we save the candidates as well for debugging the creative flow
        if final_values.get("candidates"):
            candidates_json = json.dumps([c.model_dump() for c in final_values["candidates"]], indent=2)
            (run_dir / "blackboard_final_candidates.json").write_text(candidates_json, encoding="utf-8")

        if final_values.get("job_body_json"):
            (run_dir / "final_selection.json").write_text(final_values["job_body_json"], encoding="utf-8")

        # Log specific User Context found in Shared Memory for this run
        if final_values.get("ruler_run"):
            _json_dump(run_dir / "ruler_performance.json", final_values["ruler_run"])

        print("\n===== FINAL JOB BODY =====\n", final_values.get("job_body_json") or "")
        print("\n===== FINAL CONSISTENCY REPORT =====\n", final_values.get("consistency_report_json") or "")
        print(f"\n[artifacts] wrote: {run_dir}")
        print(f"[artifacts] events: {events_path}")

        # keep return signature as you had it
        return final_values, latest, history

    finally:
        await conn.close()


In [19]:
result, latest, history = await run_example()


[Expert Active: scrape_company]

[Expert Active: generator]
  Blackboard Update: 3 candidates present.
  Sample Draft: As a Senior AI Agent Engineer, you'll be at the forefront of building and deploying intelligent systems that drive our startup's mission forward. You'll work in a fast-paced, collaborative environment...

[Expert Active: style_expert]
  Blackboard Update: 3 candidates present.
  Sample Draft: As a Senior AI Agent Engineer, you'll be at the forefront of building and deploying intelligent systems that drive our startup's mission forward. You'll work in a fast-paced, collaborative environment...

[Expert Active: curator]
  Curator selected final candidate (Size: 2059 chars)
  RULER Ranking: Best Score = 0.93

[Expert Active: persist]

===== FINAL JOB BODY =====
 {
  "job_description": "Join a dynamic AI-driven startup in Switzerland and lead the development of cutting-edge AI agents. As a Senior AI Agent Engineer, you'll design, build, and orchestrate intelligent systems